In [539]:
import numpy as np
import re 
from operator import itemgetter

In [540]:
def def_matriz(seq_vert, seq_horiz):

    num_col = len(seq_vert) + 1
    num_lin = len(seq_horiz) + 1
    matriz = np.zeros((num_col, num_lin))

    return matriz

In [541]:

def read_arq():
    with open('input.txt', 'r') as arquivo:
        linhas = arquivo.readlines()
    seq_vert = linhas[0].strip()
    seq_horiz = linhas[1].strip()
    GAP = linhas[2].strip()
    mismatch = linhas[3].strip()
    match = linhas[4].strip()
    return seq_vert, seq_horiz, GAP, mismatch, match


In [542]:
def comp_sequence(matriz, seq_vert, seq_horiz, mismatch, match, i, j):
    if seq_vert[i-1] == seq_horiz[j-1]:
        return int(matriz[i-1][j-1]) + int(match)
    else:
        return int(matriz[i-1][j-1]) + int(mismatch)
        

In [543]:
def smith_waterman(matriz, seq_vert, seq_horiz, GAP, mismatch, match):
    GAP = int(GAP)
    mismatch = int(mismatch)
    match = int(match)
    num_lin = len(matriz)
    num_col = len(matriz[0])

    sum = 0

    matriz_back = def_matriz(seq_vert, seq_horiz)
    #set GAP first column
    for i in range(num_lin):
        for j in range(num_col):
            if j == 0 and i != 0:
                matriz_back[i][j] = 2
    
    #set GAP first line
    for i in range(num_lin):
        for j in range(num_col):
            if i == 0 and j != 0:
                matriz_back[i][j] = 2

    #set GAP first column
    for i in range(num_lin):
        for j in range(num_col):
            if j == 0 and i != 0:
                sum = sum + GAP
                matriz[i][j] = sum

    sum = 0
    
    #set GAP first line
    for i in range(num_lin):
        for j in range(num_col):
            if i == 0 and j != 0:
                sum = sum + GAP
                matriz[i][j] = sum

    for i in range(1, num_lin):
        for j in range(1, num_col):

                diag = comp_sequence(matriz, seq_vert, seq_horiz, mismatch, match, i, j)
                left = matriz[i][j] + matriz[i][j-1] + GAP
                up = matriz[i][j] + matriz[i-1][j] + GAP

                values = [diag, left, up]
                max_index, max_value = max(enumerate(values), key=lambda x: x[1])

                if max_index == 0:      #diag
                    matriz_back[i][j] = 0 
                elif max_index == 1:    #left
                    matriz_back[i][j] = 1
                elif max_index == 2:    #up
                    matriz_back[i][j] = 2

                matriz[i][j] = max_value

    return matriz, matriz_back
            

In [544]:
def backtrace(matriz_back, seq_vert, seq_horiz):
    x = len(seq_horiz) - 1
    y = len(seq_vert) - 1
    i = len(matriz_back) - 1
    j = len(matriz_back[0]) - 1

    lista_vert = []
    lista_horiz = []

    while x >= 0 and y >= 0 and i >= 0 and j >= 0:
        if matriz_back[i][j] == 0:
            lista_vert.append(seq_vert[y])
            lista_horiz.append(seq_horiz[x])
            x -= 1
            y -= 1
            i -= 1
            j -= 1

        elif matriz_back[i][j] == 1:
            lista_vert.append('-')
            lista_horiz.append(seq_horiz[x])
            x -= 1
            j -= 1

        elif matriz_back[i][j] == 2:
            lista_vert.append(seq_vert[y])
            lista_horiz.append('-')
            y -= 1
            i -= 1


    while y >= 0 and x < 0:
        lista_vert.append(seq_vert[y])
        lista_horiz.append('-')
        y -= 1

    while x >= 0 and y < 0:
        lista_vert.append('-')
        lista_horiz.append(seq_horiz[x])
        x -= 1

    



            # x = 2/ y = 3
            # i = 4/ j = 3

            # x = 1/ y = 2
            # i = 3/ j = 2

            # x = 0/ y = 1
            # i = 2/ j = 1

            # x = -1/ y = 0
            # i = 1/ j =  0

 

    return lista_vert[::-1], lista_horiz[::-1]


In [545]:
# teste de uso das variáveis
print("Vertical:", read_arq()[0])
print("Horizontal:", read_arq()[1])
print("GAP:", read_arq()[2])
print("Mismatch:", read_arq()[3])
print("Match:", read_arq()[4])

matriz_initial = def_matriz(read_arq()[0], read_arq()[1])
matriz = smith_waterman(matriz_initial, read_arq()[0], read_arq()[1], read_arq()[2], read_arq()[3], read_arq()[4])
matriz_new = backtrace(matriz[1], read_arq()[0], read_arq()[1])

print(matriz[0])
print("\n")
print(matriz[1])
print("\n")
print(matriz_new[0])
print("\n")
print(matriz_new[1])
print("\n")


Vertical: UGCACGGCCAAACCGUACAGUCCUGCACGCAGCGCCUUGGUCCUGUGCCUAAGCGAUGGGGGUUGAAACACUUACACAAAAAACCGUCUCCAUUAUCUCAGUUCUCCAGGCUCCACUCUUGCUUUUGGAGAGCAUAAAUCUUGGUAACAUAAAGUUUCGGGUGGCGACGCUCGCAGUUACGCUACGGUAGCGUAGCGUGU
Horizontal: UGCACGGCCAAACCGUACAGUCCUGCACGCCUUGGUCCUGUGCCUAAGCGAUGGGGGUUGAAACACUAGACUUACACAAAAAACCGUCUCCAUUAUCUGUUCUCAGUUCUCCAGGCUCUUGCUUUUGGAGAGC
GAP: -2
Mismatch: -1
Match: 3
[[   0.   -2.   -4. ... -262. -264. -266.]
 [  -2.    3.    1. ... -257. -259. -261.]
 [  -4.    1.    6. ... -252. -254. -256.]
 ...
 [-396. -391. -386. ...  220.  225.  230.]
 [-398. -393. -388. ...  218.  223.  228.]
 [-400. -395. -390. ...  216.  221.  226.]]


[[0. 2. 2. ... 2. 2. 2.]
 [2. 0. 1. ... 1. 1. 1.]
 [2. 2. 0. ... 1. 0. 1.]
 ...
 [2. 0. 2. ... 0. 2. 2.]
 [2. 2. 0. ... 0. 0. 2.]
 [2. 0. 2. ... 0. 2. 2.]]


['U', 'G', 'C', 'A', 'C', 'G', 'G', 'C', 'C', 'A', 'A', 'A', 'C', 'C', 'G', 'U', 'A', 'C', 'A', 'G', 'U', 'C', 'C', 'U', 'G', 'C', 'A', 'C', 'G', 'C', 'A', 'G', 'C', 'G', 'C', 'C', 'U', 'U'